In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import math
from sklearn.preprocessing import StandardScaler
import random
import sklearn
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
desharnais=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\desharnais_dataset.csv",header=None,index_col=0)
columns_desharnais=['TeamExp','ManagerExp','YearEnd','Length','Effort','Transactions','Entities','PointsAdjust','Envergure','PointsNonAjust','Language']
desharnais.set_axis(columns_desharnais,axis='columns',inplace=True)
desharnais.rename_axis("Features", axis=1,inplace=True)
desharnais.rename_axis("Projects", axis=0,inplace=True)
dataset = desharnais

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)
# Split data into train and test sets

# Create a classifier
estimator = SVR(kernel='linear')

# Create SFS model
sfs = SequentialFeatureSelector(estimator,n_features_to_select='auto', direction='forward', cv=5 , scoring='neg_mean_absolute_error')

# Fit SFS model on training data
sfs.fit(X, y)

# Transform the data to selected features
X_selected = sfs.transform(X)


#=====================================================================================================================

scaler_X = StandardScaler()
X_selected = scaler_X.fit_transform(X_selected)


# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
    'neurons_input': (10, 50),
    'neurons_hidden': (10, 50),
    'num_hidden_layers': (1, 5),
    'learning_rate': (1e-5, 1e-2),
    'batch_size': (8, 32),
    'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)

    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 52ms/step
| 1         | -5.64e+03 | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 42ms/step
| 2         | -2.492e+0 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 40ms/step
| 3         | -5.772e+0 | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 36ms/step
| 4         | -5.753e+0 | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 0s 32ms/step
| 5         | -5.767e+0 | 18.95     | 16.78     | 0.002005  | 30.57     | 33.7      | 1.186     |
1/1 [==============================] - 0s 42ms/step
| 

1/1 [==============================] - 0s 47ms/step
| 32        | -4.711e+0 | 28.45     | 6.313     | 0.01      | 50.0      | 10.0      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 33        | -5.773e+0 | 8.0       | 20.0      | 1e-05     | 24.61     | 50.0      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 34        | -2.576e+0 | 32.0      | 20.0      | 0.01      | 50.0      | 22.65     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 35        | -5.773e+0 | 19.47     | 20.0      | 1e-05     | 50.0      | 19.92     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 36        | -5.773e+0 | 15.84     | 20.0      | 1e-05     | 20.69     | 33.45     | 5.0       |
1/1 [==============================] - 0s 32ms/step
| 37        | -2.961e+0 | 8.908     | 18.72     | 0.01      | 45.21     | 13.42     | 1.579     |
1/1 [==============================] - 0s 43ms/step
| 38        | -5.593e+0 | 15.05     | 19.01     

1/1 [==============================] - 0s 42ms/step
| 29        | -2.499e+0 | 19.12     | 9.833     | 0.009997  | 40.51     | 47.35     | 4.912     |
1/1 [==============================] - 0s 43ms/step
| 30        | -2.781e+0 | 8.377     | 16.28     | 0.007496  | 38.55     | 13.84     | 4.784     |
1/1 [==============================] - 0s 40ms/step
| 31        | -5.458e+0 | 22.25     | 10.64     | 0.01      | 39.78     | 48.13     | 3.875     |
1/1 [==============================] - 0s 46ms/step
| 32        | -2.783e+0 | 19.04     | 7.334     | 0.01      | 40.78     | 45.88     | 5.0       |
1/1 [==============================] - 0s 42ms/step
| 33        | -3.046e+0 | 16.97     | 12.89     | 0.01      | 40.48     | 46.9      | 4.518     |
1/1 [==============================] - 0s 46ms/step
| 34        | -2.279e+0 | 9.067     | 14.41     | 0.007213  | 37.68     | 10.57     | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 35        | -2.391e+0 | 11.77     | 15.22     

1/1 [==============================] - 0s 48ms/step
| 26        | -2.756e+0 | 8.0       | 6.404     | 0.01      | 21.95     | 42.7      | 5.0       |
1/1 [==============================] - 0s 50ms/step
| 27        | -2.623e+0 | 8.0       | 20.0      | 0.01      | 17.74     | 43.9      | 5.0       |
1/1 [==============================] - 0s 48ms/step
| 28        | -2.426e+0 | 8.0       | 11.81     | 0.01      | 17.93     | 48.69     | 5.0       |
1/1 [==============================] - 0s 332ms/step
| 29        | -2.577e+0 | 8.672     | 16.06     | 0.008641  | 24.11     | 49.45     | 3.64      |
1/1 [==============================] - 0s 54ms/step
| 30        | -2.5e+03  | 8.0       | 8.069     | 0.01      | 30.32     | 50.0      | 5.0       |
1/1 [==============================] - 0s 50ms/step
| 31        | -2.757e+0 | 15.97     | 8.223     | 0.01      | 25.76     | 50.0      | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 32        | -2.82e+03 | 13.14     | 15.78    

1/1 [==============================] - 0s 51ms/step
| 23        | -3.314e+0 | 15.37     | 6.59      | 0.01      | 34.26     | 44.73     | 5.0       |
1/1 [==============================] - 0s 58ms/step
| 24        | -4.642e+0 | 10.32     | 5.756     | 0.01      | 37.02     | 45.61     | 5.0       |
1/1 [==============================] - 0s 44ms/step
| 25        | -6.638e+0 | 17.23     | 5.569     | 0.005112  | 29.82     | 46.87     | 3.004     |
1/1 [==============================] - 0s 50ms/step
| 26        | -3.542e+0 | 17.01     | 7.951     | 0.01      | 37.37     | 43.01     | 5.0       |
1/1 [==============================] - 0s 37ms/step
| 27        | -6.653e+0 | 15.98     | 7.301     | 0.001634  | 39.02     | 44.51     | 1.396     |
1/1 [==============================] - 0s 46ms/step
| 28        | -6.625e+0 | 19.43     | 5.245     | 0.003912  | 34.3      | 41.95     | 4.057     |
1/1 [==============================] - 0s 46ms/step
| 29        | -3.46e+03 | 16.68     | 7.421     

1/1 [==============================] - 0s 53ms/step
| 20        | -4.216e+0 | 22.49     | 20.0      | 0.01      | 16.05     | 35.56     | 5.0       |
1/1 [==============================] - 0s 36ms/step
| 21        | -7.559e+0 | 8.0       | 20.0      | 1e-05     | 10.0      | 32.1      | 1.0       |
1/1 [==============================] - 0s 59ms/step
| 22        | -4.259e+0 | 17.58     | 13.55     | 0.01      | 12.32     | 38.3      | 5.0       |
1/1 [==============================] - 0s 43ms/step
| 23        | -7.499e+0 | 17.13     | 19.57     | 0.002542  | 13.94     | 44.88     | 3.243     |
1/1 [==============================] - 0s 53ms/step
| 24        | -3.736e+0 | 17.36     | 17.85     | 0.01      | 13.13     | 32.29     | 5.0       |
1/1 [==============================] - 0s 50ms/step
| 25        | -3.988e+0 | 8.0       | 10.99     | 0.01      | 41.95     | 10.0      | 5.0       |
1/1 [==============================] - 0s 51ms/step
| 26        | -7.559e+0 | 8.0       | 11.99     

In [4]:
print(sfs.get_feature_names_out())

['x0' 'x1' 'x3' 'x8' 'x9']
